In [1]:
import sys
sys.path.insert(0, '/root/mounted/projects/qpair/')

In [2]:
from config import *

In [3]:
# 加载模块
import pandas as pd
import pickle
import os
import numpy as np
from add_vectors_to_qpairs import add_wv_to_qpairs
from keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0521/intermediate/'
KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0523/publish/'
KN_PAIRS_SAMPLED_TRAIN_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'train.csv')
KN_PAIRS_SAMPLED_TEST_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'test.csv')

In [5]:
kn_pairs_sampled_train = pd.read_csv(KN_PAIRS_SAMPLED_TRAIN_CSV)
kn_pairs_sampled_train

,label,q1,q2
0,1,Q173098,Q208402
1,1,Q322606,Q105226
2,1,Q168520,Q392700
3,1,Q269823,Q353449
4,1,Q287705,Q280416
5,1,Q725259,Q312842
6,1,Q672129,Q358618
7,1,Q395573,Q625513
8,1,Q705867,Q543111
9,1,Q425933,Q476211


In [6]:
# kn_pairs_sampled_train.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_train.columns = ['label', 'qid1', 'qid2']
kn_pairs_sampled_train

,label,qid1,qid2
0,1,Q173098,Q208402
1,1,Q322606,Q105226
2,1,Q168520,Q392700
3,1,Q269823,Q353449
4,1,Q287705,Q280416
5,1,Q725259,Q312842
6,1,Q672129,Q358618
7,1,Q395573,Q625513
8,1,Q705867,Q543111
9,1,Q425933,Q476211


In [7]:
kn_pairs_sampled_test = pd.read_csv(KN_PAIRS_SAMPLED_TEST_CSV)
kn_pairs_sampled_test

,q1,q2
0,Q151557,Q293789
1,Q312362,Q489267
2,Q183757,Q521124
3,Q569210,Q202128
4,Q471665,Q658625
5,Q559930,Q353992
6,Q144120,Q142113
7,Q480774,Q591009
8,Q475024,Q591843
9,Q278079,Q384012


In [8]:
# kn_pairs_sampled_test.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_test.columns = [ 'qid1', 'qid2']
kn_pairs_sampled_test

,qid1,qid2
0,Q151557,Q293789
1,Q312362,Q489267
2,Q183757,Q521124
3,Q569210,Q202128
4,Q471665,Q658625
5,Q559930,Q353992
6,Q144120,Q142113
7,Q480774,Q591009
8,Q475024,Q591843
9,Q278079,Q384012


In [9]:
with open(QUESTION_PKL, 'rb') as f:
    qs = pickle.load(f)

qs

,qid,words,chars
0,Q000000,W05733 W05284 W09158 W14968 W07863,L1128 L1861 L2218 L1796 L1055 L0847 L2927
1,Q000001,W17378 W17534 W03249 W01490 W18802,L2214 L1980 L0156 L1554 L2218 L1861 L3019 L010...
2,Q000002,W17378 W08158 W20171 W11246 W14759,L2214 L2350 L2568 L1969 L2168 L0694 L3012 L256...
3,Q000003,W11385 W14103 W02556 W13157 W09749,L0762 L2172 L1971 L2034 L2039 L2271 L1346 L223...
4,Q000004,W17508 W18238 W02952 W18103,L0018 L2321 L1346 L3019 L0104 L0902 L0354 L0362
5,Q000005,W17378 W12908 W19355 W08410 W11922 W14496 W20487,L2214 L1331 L0016 L1132 L2237 L1900 L0578 L195...
6,Q000006,W04346 W17378 W06112 W05733 W18238 W05284 W118...,L2323 L1526 L2214 L0358 L1849 L1128 L1861 L301...
7,Q000007,W16696 W02556 W16564 W03914 W18103,L0366 L2028 L2034 L2039 L0605 L1824 L2292 L015...
8,Q000008,W17378 W11385 W18238 W13157 W12244 W16319 W18448,L2214 L0762 L2172 L3019 L0104 L2271 L1346 L014...
9,Q000009,W14113 W16319 W18399 W03472 W11399 W07863 W18103,L0947 L2323 L2582 L0143 L2600 L2272 L1872 L123...


In [10]:
with open(WEM_PKL, 'rb') as f:
    wmap = pickle.load(f)

w2i = wmap['word2index']
w2i

{'W00098': 14770,
 'W16604': 15866,
 'W10436': 13715,
 'W06113': 3063,
 'W13217': 9155,
 'W20796': 9167,
 'W02762': 11533,
 'W13876': 3678,
 'W15917': 672,
 'W12689': 16296,
 'W00739': 14724,
 'W20306': 18899,
 'W20564': 3081,
 'W08142': 12618,
 'W11065': 15515,
 'W18933': 20630,
 'W17039': 14965,
 'W20435': 4366,
 'W20080': 8831,
 'W18106': 5439,
 'W15394': 6965,
 'W12557': 3122,
 'W19316': 5595,
 'W17530': 11952,
 'W08889': 11964,
 'W01371': 15211,
 'W16757': 964,
 'W06990': 14830,
 'W03274': 9700,
 'W17619': 16136,
 'W08898': 15004,
 'W04714': 521,
 'W12135': 6339,
 'W15090': 19566,
 'W15962': 3599,
 'W09715': 1398,
 'W01395': 6223,
 'W10887': 3829,
 'W00646': 12899,
 'W08110': 18904,
 'W14144': 14202,
 'W20688': 12140,
 'W15468': 15265,
 'W09409': 14997,
 'W02734': 7969,
 'W12626': 1258,
 'W09165': 20062,
 'W09300': 3374,
 'W11645': 142,
 'W06914': 2983,
 'W06003': 7368,
 'W09862': 16704,
 'W17491': 9874,
 'W10347': 976,
 'W06644': 12792,
 'W18364': 18253,
 'W17607': 4630,
 'W14704

In [11]:
with open(CEM_PKL, 'rb') as f:
    cmap = pickle.load(f)

c2i = cmap['char2index']
c2i

{'L0042': 215,
 'L2477': 1342,
 'L0332': 2189,
 'L1314': 2370,
 'L2678': 2540,
 'L2343': 418,
 'L2662': 2957,
 'L1151': 958,
 'L0034': 2259,
 'L0260': 481,
 'L0735': 1432,
 'L0308': 991,
 'L0981': 2588,
 'L1819': 1917,
 'L0744': 902,
 'L1853': 649,
 'L2379': 2374,
 'L1730': 2315,
 'L2471': 2810,
 'L1291': 1974,
 'L1898': 710,
 'L0662': 182,
 'L0600': 1031,
 'L0313': 2303,
 'L2799': 805,
 'L0318': 31,
 'L2643': 886,
 'L0014': 1061,
 'L1576': 499,
 'L0802': 2470,
 'L2437': 319,
 'L0329': 233,
 'L0958': 216,
 'L2052': 2745,
 'L0946': 1528,
 'L2946': 246,
 'L0966': 1486,
 'L0931': 1517,
 'L1625': 88,
 'L1401': 2553,
 'L1676': 2419,
 'L1717': 1614,
 'L0441': 2763,
 'L2251': 2278,
 'L0414': 1048,
 'L0237': 349,
 'L1269': 1087,
 'L1781': 1864,
 'L1725': 735,
 'L0583': 2356,
 'L2556': 2049,
 'L0474': 500,
 'L0716': 804,
 'L1532': 1847,
 'L2205': 2090,
 'L0685': 1911,
 'L1757': 1379,
 'L1620': 1326,
 'L1914': 894,
 'L1707': 1897,
 'L0552': 819,
 'L0572': 2990,
 'L1504': 2996,
 'L1067': 45,
 'L1

In [12]:
kn_pairs_sampled_wids_cids_train = add_wv_to_qpairs(kn_pairs_sampled_train, qs, w2i, c2i, kn_pairs_sampled_train.shape[0])
kn_pairs_sampled_wids_cids_train

208024it [05:33, 624.11it/s]                            


,label,qid1,qid2,words1,words2,chars1,chars2
0,1,Q173098,Q208402,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Q322606,Q105226,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,Q168520,Q392700,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,Q269823,Q353449,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,Q287705,Q280416,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1,Q725259,Q312842,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1,Q672129,Q358618,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,1,Q395573,Q625513,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,1,Q705867,Q543111,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,1,Q425933,Q476211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
kn_pairs_sampled_wids_cids_test = add_wv_to_qpairs(kn_pairs_sampled_test, qs, w2i, c2i, kn_pairs_sampled_test.shape[0])
kn_pairs_sampled_wids_cids_test.head()

50855it [01:24, 604.34it/s]                           


,qid1,qid2,words1,words2,chars1,chars2
0,Q151557,Q293789,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q312362,Q489267,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q183757,Q521124,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q569210,Q202128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q471665,Q658625,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
kn_pairs_sampled_test.head()

,qid1,qid2
0,Q151557,Q293789
1,Q312362,Q489267
2,Q183757,Q521124
3,Q569210,Q202128
4,Q471665,Q658625


In [16]:
# 保存
KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'train.pkl')
KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'test.pkl')
with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_train, f)

with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_test, f)
